In [1]:
!pip install -U ginza ja-ginza

# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

# 作業フォルダへの移動を行います。
# もしアップロードした場所が異なる場合は作業場所を変更してください。
import os
os.chdir('/content/drive/MyDrive/ai_app_dev/5章') #ここを変更

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 MB 10.5 MB/s eta 0:00:00
Mounted at /content/drive


## 単語集計で文章を特徴量化してみよう

In [2]:
import pandas as pd
data = pd.read_csv("data/survey.csv")
print(len(data))
data.head()

86


,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている(AA駅),1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる（待機児童なし）,4
4,2019/1/6,駅前商店街が寂しい,2


In [3]:
import spacy

nlp = spacy.load('ja_ginza')
tg_pos = 'NOUN'

data = data.dropna()
data.reset_index(drop=True, inplace=True)
print(len(data))

84


In [4]:
all_words_df = pd.DataFrame()
for i in range(len(data)):
  doc = nlp(data['comment'][i])
  word_arr = []
  for token in doc:
    if token.pos_ in tg_pos:
      word_arr.append(token.lemma_)
  words_df = pd.DataFrame({"words":word_arr, "count":len(word_arr)*[1]})
  words_df = words_df.groupby("words").sum().T
  words_df.reset_index(drop=True, inplace=True)
  words_df.index = [data['comment'][i]]
  all_words_df = pd.concat([all_words_df, words_df])
all_words_df = all_words_df.fillna(0)
all_words_df.head()

words,AA,若者,駅,駅前,場所,子育て,支援,なし,保育園,待機児童,...,糞害,野良猫,とき,高速道路,まち,不安,サポート,高齢者,校庭,遊び道具
駅前に若者が集まっている(AA駅),1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
スポーツできる場所があるのが良い,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
子育て支援が嬉しい,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
保育園に入れる（待機児童なし）,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
駅前商店街が寂しい,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 単語分散表現による特徴量化を体験しよう

In [5]:
nlp = spacy.load('ja_ginza')
doc = nlp("男")
for token in doc:
  print(token)
  print(token.vector)

男
[ 0.15933658 -0.0601605   0.24383044 -0.22848123 -0.16004498  0.09069587
 -0.05276782 -0.16438423 -0.07487892 -0.02849463  0.19212672 -0.28361022
 -0.12933674  0.07361041  0.06852581 -0.11502243 -0.1906645  -0.0874429
 -0.03073838  0.13075289 -0.25983295 -0.0621451  -0.01141335  0.14354426
 -0.3612975  -0.0302887  -0.03045526 -0.21420372 -0.18271627  0.2238288
  0.12131057  0.1455816  -0.04027628  0.09775832 -0.01657178  0.13552241
 -0.2561508  -0.13380808  0.02541774 -0.1111074   0.03344875  0.3130034
 -0.11582399 -0.07680997  0.0101055  -0.25840396 -0.01797279 -0.0837252
 -0.02153593  0.16043833  0.0545024  -0.13318448  0.20580207  0.01426451
 -0.12973121  0.06106999  0.04306051  0.12404229 -0.06068777  0.13532683
 -0.06737218 -0.05633878 -0.0249875  -0.06449635  0.11541229 -0.02239965
 -0.11697876  0.04648037 -0.05727889  0.08222914 -0.13934301  0.06897237
  0.0649143   0.18646991  0.02686984 -0.0396568  -0.16417246  0.0704788
  0.09646127  0.09645716  0.11937089 -0.34765097 -0.19

In [6]:
doc1 = nlp("男")
for token in doc1:
  vec1 = token.vector

doc2 = nlp("男性")
for token in doc2:
  vec2 = token.vector

doc3 = nlp("遊び")
for token in doc3:
  vec3 = token.vector

In [7]:
import numpy as np

sim12 = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
sim13 = np.dot(vec1, vec3) / (np.linalg.norm(vec1) * np.linalg.norm(vec3))
print(sim12)
print(sim13)

0.6332923
0.23904727


In [8]:
print(doc1.similarity(doc2))
print(doc1.similarity(doc3))

0.6332922811605247
0.23904726112491734


In [9]:
doc = nlp("私は旅行に行きます")
print(doc.vector)

[-6.24577291e-02 -2.63049871e-01  1.09813795e-04 -2.29318127e-01
 -7.87388906e-02  1.17492191e-01 -1.52793273e-01 -8.33533928e-02
 -1.59250751e-01  3.85920447e-03  9.13133845e-02 -6.96304664e-02
  2.34625395e-02  6.76364526e-02 -9.28347185e-02 -2.63118207e-01
 -1.34960040e-01 -1.04109347e-01 -1.62048742e-01  1.08887255e-01
 -1.12728246e-01  1.60532147e-01  6.45211339e-02  9.62401256e-02
 -1.24947071e-01 -1.21119820e-01 -1.69587806e-01  5.30675352e-02
  2.69064400e-02  1.14622228e-01  7.19811395e-02 -3.07314470e-02
  6.08067103e-02  2.11805835e-01  1.96584407e-02  5.63849397e-02
 -9.86932516e-02  1.23430289e-01 -7.88782313e-02  6.73780665e-02
  2.62418613e-02  2.34157755e-03 -5.48435487e-02 -4.26859669e-02
 -1.08611465e-01 -3.17440964e-02  1.23132288e-01  3.49572301e-03
 -1.35607705e-01  5.39614260e-02  7.58203119e-02 -1.38307912e-02
 -1.31406682e-02  7.02086538e-02  9.00378600e-02 -1.28202185e-01
  2.29950398e-02  1.20312832e-01 -5.22284843e-02 -2.66706254e-02
  8.77813771e-02  7.73132

In [10]:
doc1 = nlp("私は旅行に行きます")
vec1 = doc1.vector

doc2 = nlp("私は遊びに行きます")
vec2 = doc2.vector

doc3 = nlp("父は数学が得意である")
vec3 = doc3.vector

In [11]:
sim12 = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
sim13 = np.dot(vec1, vec3) / (np.linalg.norm(vec1) * np.linalg.norm(vec3))
print(sim12)
print(sim13)

0.9312169
0.65789765
